Code for Code Explaination


In [ ]:
import requests
import json
import os
from google.colab import userdata
from google.colab import auth
import gspread
from google.auth import default

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

def get_openai_response(prompt_text: str,  api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str, api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})

    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

def add_to_google_sheet(data):
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Create a new spreadsheet or open an existing one
    try:
        sh = gc.open("LLM_Code_Analysis")
    except gspread.exceptions.SpreadsheetNotFound:
        sh = gc.create("LLM_Code_Analysis")
        # Share with yourself to have permission to view it in Google Drive
        sh.share(userdata.get('EMAIL'), perm_type='user', role='writer')

    worksheet = sh.sheet1

    # Append data to the next available row
    worksheet.append_row(data)
def get_prompts_from_sheet():
    # Authenticate and access the sheet
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Open the spreadsheet and the "Explanation" sheet
    sheet = gc.open("ASE Tool Dataset").worksheet("Explanation")

    # Fetch the prompts from the "Prompt to Code" column
    prompt_data = sheet.col_values(5)  # This fetches the entire column

    # Skip the header and return the prompts
    return prompt_data[1:31]

# --- Main Logic ---

def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    prompts = get_prompts_from_sheet()



    print("\nGetting responses from LLMs...\n")

 for i, user_prompt in enumerate(prompts, start=1):
    # Get response from GPT-4o
    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, OPENAI_API_KEY)
    print(gpt_response[1])
    print("-" * 30)
    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt,  DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)

    # Add results to Google Sheet
    sheet_data = [f"Prompt {i}", gpt_response, deepseek_response, gemini_response]
    add_to_google_sheet(sheet_data)
    print("Results added to Google Sheet.")


if __name__ == "__main__":
    main()

Welcome to the LLM Code Analyzer!
------------------------------

Getting responses from LLMs...

--- GPT-4o Response ---
### Context of the Project Repository
In the context of the Keel project repository, `Sobel.java` implements an edge detection algorithm using the Sobel filter. This code processes height maps in greyscale to enhance and outline features within the image, contributing to any image processing tasks that require edge detection or analysis.

### Explanation of the Code
The `Sobel.java` file is part of the `com.harium.keel.effect.height` package and implements edge detection using the Sobel filter. As an implementation of the `Effect` interface, it takes a grayscale height map as input (`ImageSource`) and returns an image highlighting edges (`MatrixSource`). The algorithm iterates over pixel values, applying the Sobel operator to compute the gradient magnitudes in both x and y directions. This involves a convolution approach with pre-defined Sobel matrices to detect int

Code for Code Quality Metrics



In [ ]:
import requests
import json
import os
from google.colab import userdata
from google.colab import auth
import gspread
from google.auth import default


OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# API Endpoints
OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# --- Helper Functions for LLM Interactions ---

def get_openai_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str,  api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})


    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

def add_to_google_sheet(data):
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Create a new spreadsheet or open an existing one
    try:
        sh = gc.open("LLM_Code_Analysis")
    except gspread.exceptions.SpreadsheetNotFound:
        sh = gc.create("LLM_Code_Analysis")
        # Share with yourself to have permission to view it in Google Drive
        sh.share(userdata.get('EMAIL'), perm_type='user', role='writer')

    worksheet = sh.sheet1

    # Append data to the next available row
    worksheet.append_row(data)
def get_prompts_from_sheet():
    # Authenticate and access the sheet
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)


    sheet = gc.open("ASE Tool Dataset").worksheet("Code Metics")

    # Fetch the prompts from the "Prompt to Code" column
    prompt_data = sheet.col_values(5)  # This fetches the entire column

    # Skip the header and return the prompts
    return prompt_data[1:31]

# --- Main Logic ---


def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    prompts = get_prompts_from_sheet()



    print("\nGetting responses from LLMs...\n")

 for i, user_prompt in enumerate(prompts, start=1):
    # Get response from GPT-4o
    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, OPENAI_API_KEY)
    print(gpt_response[1])
    print("-" * 30)
    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt,  DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)

    # Add results to Google Sheet
    sheet_data = [f"Prompt {i}", gpt_response, deepseek_response, gemini_response]
    add_to_google_sheet(sheet_data)



if __name__ == "__main__":
    main()

Welcome to the LLM Code Analyzer!
------------------------------

Getting responses from LLMs...

--- GPT-4o Response ---
Cyclomatic Complexity: 22  
Maintainability Index: 90  
Vulnerability Category: Memory Corruption
------------------------------


Code for Code Refactoring


In [ ]:
import requests
import json
import os
from google.colab import userdata
from google.colab import auth
import gspread
from google.auth import default


OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# API Endpoints
OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# --- Helper Functions for LLM Interactions ---

def get_openai_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str,  api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})


    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

def add_to_google_sheet(data):
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Create a new spreadsheet or open an existing one
    try:
        sh = gc.open("LLM_Code_Analysis")
    except gspread.exceptions.SpreadsheetNotFound:
        sh = gc.create("LLM_Code_Analysis")
        # Share with yourself to have permission to view it in Google Drive
        sh.share(userdata.get('EMAIL'), perm_type='user', role='writer')

    worksheet = sh.sheet1

    # Append data to the next available row
    worksheet.append_row(data)
def get_prompts_from_sheet():
    # Authenticate and access the sheet
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)


    sheet = gc.open("ASE Tool Dataset").worksheet("Refactoring")

    # Fetch the prompts from the "Prompt to Code" column
    prompt_data = sheet.col_values(5)  # This fetches the entire column

    # Skip the header and return the prompts
    return prompt_data[1:31]
# --- Main Logic ---


def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    prompts = get_prompts_from_sheet()



    print("\nGetting responses from LLMs...\n")

 for i, user_prompt in enumerate(prompts, start=1):
    # Get response from GPT-4o

    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, OPENAI_API_KEY)
    print(gpt_response[1])
    print("-" * 30)
    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt,  DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)

    # Add results to Google Sheet
    sheet_data = [f"Prompt {i}", gpt_response, deepseek_response, gemini_response]
    add_to_google_sheet(sheet_data)



if __name__ == "__main__":
    main()

Welcome to the LLM Code Analyzer!
------------------------------

Getting responses from LLMs...

--- GPT-4o Response ---
```java
public Object streamLoad(HttpServletRequest request,
                         @PathVariable(value = DB_KEY) String db,
                         @PathVariable(value = TABLE_KEY) String table) {
    if (isGroupCommitEnabled(request)) { // Extracted repeated logic into a separate method
        try {
            String[] pair = new String[]{db, table};
            if (isGroupCommitBlock(pair)) {
                return new RestBaseResult("insert table " + pair[1] + " is blocked on schema change");
            }
        } catch (Exception e) {
            // handle
        }
    }
    return null; // Added return null to ensure method always returns an Object
}

@RequestMapping(path = "/api/_http_stream", method = RequestMethod.PUT)
public Object streamLoadWithSql(HttpServletRequest request, HttpServletResponse response) {
    String sql = request.getHeader("sql"